In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

# 1. 데이터 셋

In [2]:
import pandas as pd
df = pd.read_csv('c:/taro/ai/source/01_python/data/ch13_df(결측치중위값으로대체).csv')
df.head()

,지역명,평당분양가격,연도,월
0,서울,18189.0,2013,12
1,부산,8111.0,2013,12
2,대구,8080.0,2013,12
3,인천,10204.0,2013,12
4,광주,6098.0,2013,12


- 지역명2 : 지역명필드를 라벨인코딩하여 추가
- 독립변수 : 지역명2, 연도, 월
- 타겟변수 : 평당분양가격
- 독립변수와 타겟변수의 스케일 조정
    * 정규화(MinMaxScaler) 작업 후 : 지역명2m, 연도m, 월m
    * 표준화(StandardScaler) 작업 후 : 지역명2s, 연도s, 월s
    => 지역명, 연도, 월, 지역명2, 지역명2m, 연도m, 월m, 지역명2s, 연도s, 월s

# 2. 지역명의 라벨인코딩
- 지역명2

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# le.fit(df['지역명'])
# le.transform(df['지역명'])
df['지역명2'] = le.fit_transform(df['지역명']) # 문자데이터를 가나다순으로 정렬후 숫자로 변환
df.head()

,지역명,평당분양가격,연도,월,지역명2
0,서울,18189.0,2013,12,8
1,부산,8111.0,2013,12,7
2,대구,8080.0,2013,12,5
3,인천,10204.0,2013,12,11
4,광주,6098.0,2013,12,4


# 3. MinMaxScale 스케일 조정

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
X_data = df[['지역명2', '연도', '월']].values # numpy배열로 추출한 독립변수
# values나 to_numpy()를 이용해서 데이터프레임을 numpy배열로 변환
y_data = df[['평당분양가격']].to_numpy()
X_data.shape, y_data.shape

((2176, 3), (2176, 1))

In [8]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
df[['지역명2m', '연도m', '월m']] = scaler_x.fit_transform(X_data)
df[['평당분양가격m']] = scaler_y.fit_transform(y_data)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757


# 4. standardScale 조정

In [9]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
scaler_y = StandardScaler()
df[['지역명2s', '연도s','월s']] = scaler_x.fit_transform(X_data)
df[['평당분양가격s']] = scaler_y.fit_transform(y_data)
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,연도s,월s,평당분양가격s
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,-1.875367,1.62196,1.168591
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,-1.875367,1.62196,-0.728312
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,-1.875367,1.62196,-0.734147
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,-1.875367,1.62196,-0.334363
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,-1.875367,1.62196,-1.107203


# 5. 지역명을 원핫인코딩
- 강원 0 => 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
- 서울 8 => 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0

In [10]:
loc = df.loc[:16, ['지역명', '지역명2']].sort_values(by='지역명2')['지역명'].tolist()
print(loc) # 원핫인코딩 후 열이름

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [13]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
# to_categorical()결과는 numpy 배열, 최대정수가 16이면 0~16카테고리로 원핫인코딩, 매개변수에 라벨인코딩 된 값
# pd.get_dummies()결과는 데이터프레임, 정수가 곧 카테고리가 되서 원핫인코딩, 매개변수에 문자
temp1 = to_categorical(df['지역명2']) 
temp2 = pd.get_dummies(df['지역명']).values
np.all(temp1 == temp2)

True

In [14]:
# 원핫인코딩 방법1
df[loc] = temp1 = to_categorical(df['지역명2'])
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df[loc] = df[loc].

In [18]:
# 원핫인코딩 데이터 지우기
df = df.drop(loc, axis=1)
df.head()

KeyError: "['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북'] not found in axis"

In [15]:
# 원핫인코딩 방법2
df[loc] = pd.get_dummies(df['지역명2'])
df.head()

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,18189.0,2013,12,8,0.5000,0.0,1.0,0.328198,0.000000,...,0,1,0,0,0,0,0,0,0,0
1,부산,8111.0,2013,12,7,0.4375,0.0,1.0,0.065274,-0.204124,...,1,0,0,0,0,0,0,0,0,0
2,대구,8080.0,2013,12,5,0.3125,0.0,1.0,0.064466,-0.612372,...,0,0,0,0,0,0,0,0,0,0
3,인천,10204.0,2013,12,11,0.6875,0.0,1.0,0.119878,0.612372,...,0,0,0,0,1,0,0,0,0,0
4,광주,6098.0,2013,12,4,0.2500,0.0,1.0,0.012757,-0.816497,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# 원핫인코딩 방법3
df = pd.concat([df,pd.get_dummies(df['지역명'])], axis=1)
df.sample(2)

,지역명,평당분양가격,연도,월,지역명2,지역명2m,연도m,월m,평당분양가격m,지역명2s,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
1968,전남,12074.7,2023,8,12,0.75,0.909091,0.636364,0.168683,0.816497,...,0,0,0,0,0,1,0,0,0,0
1267,강원,8144.4,2020,3,0,0.00,0.636364,0.181818,0.066146,-1.632993,...,0,0,0,0,0,0,0,0,0,0
